## Asociačné pravidlá - analýza nákupného košíka

Jeden z druhov popisnej analýzy dát sú asociačné pravidlá. Predstavujú špecifický typ popisného modelu, ktorý je aplikovateľný na transakčných dátach. To sú dáta reprezentujúce napr. nákupné, alebo iné transakcie. Riadky v nich zodpovedajú jednotlivých transakciám (napr. jednotlivým nákupom) a stĺpce potom reprezentujú jednotlivé tovary. Hodnoty v jednotlivých stĺpcovh potom indikujú, či v sa v danom nákupe niektorý z tovarov vyskytol, alebo nie.

Asociačné pravidlá potom používame na odhalenie častých kombinácií položiek v rámci množiny transakcií. Tento princíp nám teda pomôže identifikovať napr. položky, ktoré sa často nakupujú spoločne (odtiaľ aj odvodený názov - analýza nákupného košíka).

Pre hľadanie asociačných pravidiel v Pythone potrebujeme najprv doinštalovať moduly obsahujúce potrebné algoritmy pre ich hľadanie. Keďže knižnica Scikit-learn takéto neobsahuje, doinštalujeme modul `mlxtend`, ktorý obsahuje algoritmus Apriori, často používaný pre hľadanie asociačných pravidiel. 

Pre inštaláciu v distribúcii Anaconda je potrebné sa prepnúť do domovskej aplikácie (Anaconda Navigator). V záložke `Environments` potom kliknite na trojuholníkový symbol pri položke `base (root)`. V nej zvoľte možnosť `Open Terminal` a v príkazovom riadku zadajte príkaz `pip install mlxtend`. Takto nainštalujeme modul a budeme ho môcť používať v skriptoch a Jupyter notebookoch. 

In [ ]:
import pandas as pd # importujeme pandas
from mlxtend.frequent_patterns import apriori # importujeme knižnice pre apriori algoritmus
from mlxtend.frequent_patterns import association_rules # importujeme knižnicu pre asociačné pravidlá

Ako ukážkový dataset použijeme dataset Online Retail, ktorý obsahuje záznamy z transakcií nákupov rôzneho tovaru prostredníctvom internetového obchodu v rôznych krajinách. Pre demonštračné účely je dataset používaný v tomto notebooku zredukovaný. 

Dáta načítame zo súboru a po vypísaní hlavičky vidíme štruktúru datasetu:
* číslo faktúry
* identifikačné číslo tovaru
* názov tovaru
* množstvo
* dátum nákupu
* cena za jednotku tovaru
* ID zákazníka
* krajina

In [ ]:
data = pd.read_excel('../data/retail.xlsx') # načítame dáta zo súboru
data.head()

Ako vidíme, dáta nemáme v požadovanej - transakčnej - podobe. Pre tieto účely musíme dáta predspracovať a zmeniť ich štruktúru. Upravíme medzery pre atribút `Description` a vyhodíme riadky, ktoré nemajú valídnu faktúru.

In [ ]:
data['Description'] = data['Description'].str.strip() # orežeme nežiadúce medzery na začiatkoch a koncoch popisov
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # odstránime riadky, ktoré majú chýbajúcu faktúru
data.head()

In [ ]:
data.tail(30)

In [ ]:
data['InvoiceNo'] = data['InvoiceNo'].astype('str') # zakódujeme číslo faktúry ako string
data = data[~data['InvoiceNo'].str.contains('C')]

Faktúru potom použijeme ako identifikátor transakcie - vstupné dáta transformujeme do dátového rámca `basket`, ktorý bude obsahovať riadky reprezentujúce nákupy (identifikované pomocou čísla faktúry) a jednotlivé atribúty budú reprezentovať počty nakúpených tovarov. Toto realizujeme pomocou zoskupenia (`groupBy`) podľa čísla faktúry, položky a množstva. Pre selekciu dát ešte môžeme zvoliť aj atribút charakterizujúci krajinu, a môžeme tak identifikovať časté kombinácie nákupov na rôznych trhoch.
Výslednú tabuľku si môžeme pozrieť vypísaním hlavičky. 

In [ ]:
basket = (data[data['Country'] == "France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket.head(20)

Pre jednoduššiu prácu s asociačnými pravidlami ešte dáta binarizujeme - jednotlivé hodnoty atribútov nám budú iba indikovať, či bola daná položka zakúpená alebo nie (počet nebudeme brať do úvahy). Preto pomocou jednoduchej funkcie transformujeme dáta. Zároveň vyhodíme aj atribút `POSTAGE` popisujúci poštovné. 

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)
basket_sets.head()

Pre hľadanie asociačných pravidiel najprv identifikujeme tzv. frekventované položky. V tomto prípade to bude predstavovať zoznam často nakupovaných položiek (samostatne, nie spolu), ktorý bude doprevádzaný informáciou o ich podpore (`support`) - tzn. podieli transakcií, v ktorých sa daná položka vyskytuje spomedzi všetkých transakcií. Pri vytváraní zoznamu týchto položie môžeme podporu použiť ako parameter pre orezanie množstva identifikovaných vzorov.

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.13, use_colnames=True)
frequent_itemsets.head(20)

Z frekventovaných položiek potom vygenerujeme asociačné pravidlá v tvare `IF` predpoklady (`antecenteds`) `THEN` závery (`consequents`). K jednotlivým pravidlám potom knižnica vypíše aj podporu daného pravidla (podporu predpokladu, záveru a aj celého pravidla), spoľahlivosť (`confidence`) alebo `lift` pravidla. 

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=5)
rules

Pri veľkom množstve pravidiel je samozrejme možné prehľadávať a orezávať vygenerované pravidlá pomocou formovania podmienok a kritérií pre zobrazovanie pravidiel. Napr. príklad nižšie demonštruje zobrazenie len tých pravidiel, ktorých podpora je väčšia ako 0.2 a spoľahlivosť väčšia ako 0.9.

In [ ]:
rules[ (rules['support'] >= 0.13) & (rules['confidence'] >= 0.8) ]